# AzureOpenAIEmbeddings

[Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service/) 是一项云服务，可帮助您通过 OpenAI、Meta 和其他公司提供的一系列预构建和精选模型，快速开发生成式 AI 体验。

LangChain.js 支持使用 [OpenAI SDK](https://github.com/openai/openai-node) 中的新 Azure 集成功能，与 [Azure OpenAI](https://azure.microsoft.com/products/ai-services/openai-service/) 进行集成。

您可以在[此页面](https://learn.microsoft.com/azure/ai-services/openai/overview)了解更多有关 Azure OpenAI 及其与 OpenAI API 区别的信息。如果您还没有 Azure 帐户，可以[创建一个免费帐户](https://azure.microsoft.com/free/)来开始使用。

这将帮助您使用 LangChain 开始使用 AzureOpenAIEmbeddings [嵌入模型](/docs/concepts/embedding_models)。有关 `AzureOpenAIEmbeddings` 功能和配置选项的详细文档，请参阅 [API 参考文档](https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html)。


```{=mdx}

:::info

以前，LangChain.js 支持通过专用的 [Azure OpenAI SDK](https://github.com/Azure/azure-sdk-for-js/tree/main/sdk/openai/openai) 与 Azure OpenAI 集成。此 SDK 现已被弃用，取而代之的是 OpenAI SDK 中的新 Azure 集成，它允许在最新 OpenAI 模型和功能发布当天即可访问，并可在 OpenAI API 和 Azure OpenAI 之间无缝切换。

如果您正在使用已弃用的 SDK 与 Azure OpenAI 集成，请参阅[迁移指南](#migration-from-azure-openai-sdk)以更新到新 API。

:::

```

## 概述
### 集成详情

| 类 | 包 | 本地支持 | [Python 支持](https://python.langchain.com/docs/integrations/text_embedding/azureopenai/) | 包下载量 | 最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [AzureOpenAIEmbeddings](https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html) | [@langchain/openai](https://api.js.langchain.com/modules/langchain_openai.html) | ❌ | ✅ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/openai?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/openai?style=flat-square&label=%20&) |

## 准备工作

要访问 Azure OpenAI 嵌入模型，您需要创建一个 Azure 帐户，获取一个 API 密钥，并安装 `@langchain/openai` 集成包。

### 凭据

您需要部署一个 Azure OpenAI 实例。您可以按照 [此指南](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal) 在 Azure 门户中部署一个实例。

一旦您的实例运行起来，请确保您拥有实例的名称和密钥。您可以在 Azure 门户中实例的 "密钥和终结点" 部分下找到密钥。

如果您使用的是 Node.js，您可以定义以下环境变量以使用该服务：

```bash
AZURE_OPENAI_API_INSTANCE_NAME=<YOUR_INSTANCE_NAME>
AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME=<YOUR_EMBEDDINGS_DEPLOYMENT_NAME>
AZURE_OPENAI_API_KEY=<YOUR_KEY>
AZURE_OPENAI_API_VERSION="2024-02-01"
```

如果您希望自动追踪您的模型调用，还可以取消以下 [LangSmith](https://docs.smith.langchain.com/) API 密钥的注释：

```bash
# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="your-api-key"
```

### 安装

LangChain AzureOpenAIEmbeddings 集成位于 `@langchain/openai` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/openai @langchain/core
</Npm2Yarn>

:::info

您可以在 [Azure OpenAI 文档](https://learn.microsoft.com/azure/ai-services/openai/reference) 中找到支持的 API 版本列表。

:::

:::tip

如果未定义 `AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME`，则会回退到 `AZURE_OPENAI_API_DEPLOYMENT_NAME` 的值作为部署名称。对于 `AzureOpenAIEmbeddings` 构造函数中的 `azureOpenAIApiEmbeddingsDeploymentName` 参数，如果未定义，也会回退到 `azureOpenAIApiDeploymentName` 的值。

:::

```

## 实例化

现在我们可以实例化我们的模型对象并嵌入文本：

In [1]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddings = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>", // In Node.js defaults to process.env.AZURE_OPENAI_API_KEY
  azureOpenAIApiInstanceName: "<your_instance_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_INSTANCE_NAME
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME
  azureOpenAIApiVersion: "<api_version>", // In Node.js defaults to process.env.AZURE_OPENAI_API_VERSION
  maxRetries: 1,
});

## 索引与检索

嵌入模型通常用于检索增强生成（RAG）流程中，既用于数据索引，也用于后续的数据检索。有关更详细的说明，请参阅我们位于[使用外部知识教程](/docs/tutorials/#working-with-external-knowledge)下的 RAG 教程。

下面展示如何使用我们前面初始化的 `embeddings` 对象进行数据索引和检索。在此示例中，我们将使用演示版 [`MemoryVectorStore`](/docs/integrations/vectorstores/memory) 来索引和检索一个示例文档。

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## 直接使用

在底层，向量存储和检索器的实现分别通过调用 `embeddings.embedDocument(...)` 和 `embeddings.embedQuery(...)` 来为其在 `fromDocuments` 和检索器的 `invoke` 操作中使用的文本创建嵌入。

您可以直接调用这些方法，以用于您自己的使用场景。

### 嵌入单个文本

您可以使用 `embedQuery` 对查询进行嵌入以用于搜索。这将生成一个专门针对该查询的向量表示：

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
   -0.024253517, -0.0054218727,   0.048715446,   0.020580322,    0.03180832,
   0.0028770117,  -0.012367731,   0.037383243,  -0.054915592,   0.032225136,
     0.00825818,  -0.023888804,   -0.01184671,   0.012257014,   0.016294925,
    0.009254632,  0.0051353113,  -0.008889917,   0.016855022,    0.04207243,
  0.00082589936,  -0.011664353,    0.00818654,   0.029020859,  -0.012335167,
   -0.019603407,  0.0013945447,    0.05538451,  -0.011625277,  -0.008153976,
    0.038607642,   -0.03811267, -0.0074440846,   0.047647353,   -0.00927417,
    0.024201415, -0.0069230637,  -0.008538228,   0.003910912,   0.052805457,
   -0.023159374,  0.0014352495,  -0.038659744,   0.017141584,   0.005587948,
    0.007971618,  -0.016920151,    0.06658646, -0.0016916894,   0.045667473,
   -0.042202685,   -0.03983204,   -0.04160351,  -0.011729481,  -0.055905532,
    0.012543576,  0.0038848612,   0.007919516,   0.010915386,  0.0033117384,
   -0.007548289,  -0.030427614,  -0.041890074,   0.036002535,  -0.02377157

### 嵌入多个文本

您可以使用 `embedDocuments` 对多个文本进行索引嵌入。此方法的内部实现可能（但不一定）与嵌入查询的方式不同：

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
   -0.024253517, -0.0054218727,   0.048715446,   0.020580322,    0.03180832,
   0.0028770117,  -0.012367731,   0.037383243,  -0.054915592,   0.032225136,
     0.00825818,  -0.023888804,   -0.01184671,   0.012257014,   0.016294925,
    0.009254632,  0.0051353113,  -0.008889917,   0.016855022,    0.04207243,
  0.00082589936,  -0.011664353,    0.00818654,   0.029020859,  -0.012335167,
   -0.019603407,  0.0013945447,    0.05538451,  -0.011625277,  -0.008153976,
    0.038607642,   -0.03811267, -0.0074440846,   0.047647353,   -0.00927417,
    0.024201415, -0.0069230637,  -0.008538228,   0.003910912,   0.052805457,
   -0.023159374,  0.0014352495,  -0.038659744,   0.017141584,   0.005587948,
    0.007971618,  -0.016920151,    0.06658646, -0.0016916894,   0.045667473,
   -0.042202685,   -0.03983204,   -0.04160351,  -0.011729481,  -0.055905532,
    0.012543576,  0.0038848612,   0.007919516,   0.010915386,  0.0033117384,
   -0.007548289,  -0.030427614,  -0.041890074,   0.036002535,  -0.02377157

## 使用 Azure 托管标识

如果你正在使用 Azure 托管标识，可以按如下方式配置凭据：

In [ ]:
import {
  DefaultAzureCredential,
  getBearerTokenProvider,
} from "@azure/identity";
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const credentials = new DefaultAzureCredential();
const azureADTokenProvider = getBearerTokenProvider(
  credentials,
  "https://cognitiveservices.azure.com/.default"
);

const modelWithManagedIdentity = new AzureOpenAIEmbeddings({
  azureADTokenProvider,
  azureOpenAIApiInstanceName: "<your_instance_name>",
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>",
  azureOpenAIApiVersion: "<api_version>",
});


## 使用不同的域名

如果你的实例托管在非默认的 `openai.azure.com` 域名下，则需要使用替代的环境变量 `AZURE_OPENAI_BASE_PATH`。
例如，以下是如何连接到域名 `https://westeurope.api.microsoft.com/openai/deployments/{DEPLOYMENT_NAME}` 的示例：

In [ ]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddingsDifferentDomain = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>", // In Node.js defaults to process.env.AZURE_OPENAI_API_KEY
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embedding_deployment_name>", // In Node.js defaults to process.env.AZURE_OPENAI_API_EMBEDDINGS_DEPLOYMENT_NAME
  azureOpenAIApiVersion: "<api_version>", // In Node.js defaults to process.env.AZURE_OPENAI_API_VERSION
  azureOpenAIBasePath:
    "https://westeurope.api.microsoft.com/openai/deployments", // In Node.js defaults to process.env.AZURE_OPENAI_BASE_PATH
});


## 自定义头部
您可以通过传入 `configuration` 字段来指定自定义头部：

In [ ]:
import { AzureOpenAIEmbeddings } from "@langchain/openai";

const embeddingsWithCustomHeaders = new AzureOpenAIEmbeddings({
  azureOpenAIApiKey: "<your_key>",
  azureOpenAIApiInstanceName: "<your_instance_name>",
  azureOpenAIApiEmbeddingsDeploymentName: "<your_embeddings_deployment_name>",
  azureOpenAIApiVersion: "<api_version>",
  configuration: {
    defaultHeaders: {
      "x-custom-header": `SOME_VALUE`,
    },
  },
});

`configuration` 字段还接受官方 SDK 接受的其他 `ClientOptions` 参数。

**注意：** 目前无法通过此方式重写特定标头 `api-key`，它将传递 `azureOpenAIApiKey` 中的值。

## 从 Azure OpenAI SDK 迁移

如果您正在使用已弃用的 Azure OpenAI SDK 和 `@langchain/azure-openai` 包，您可以通过以下步骤将代码更新为使用新的 Azure 集成：

1. 安装新的 `@langchain/openai` 包，并移除之前的 `@langchain/azure-openai` 包：
   ```bash npm2yarn
   npm install @langchain/openai
   npm uninstall @langchain/azure-openai
   ```
2. 更新您的导入语句，使用 `@langchain/openai` 包中的新 `AzureOpenAIEmbeddings` 类：
   ```typescript
   import { AzureOpenAIEmbeddings } from "@langchain/openai";
   ```
3. 更新您的代码以使用新的 `AzureOpenAIEmbeddings` 类并传入所需的参数：

   ```typescript
   const model = new AzureOpenAIEmbeddings({
     azureOpenAIApiKey: "<your_key>",
     azureOpenAIApiInstanceName: "<your_instance_name>",
     azureOpenAIApiEmbeddingsDeploymentName:
       "<your_embeddings_deployment_name>",
     azureOpenAIApiVersion: "<api_version>",
   });
   ```

   请注意，现在构造函数需要 `azureOpenAIApiInstanceName` 参数而不是 `azureOpenAIEndpoint` 参数，并新增了 `azureOpenAIApiVersion` 参数用于指定 API 版本。

   - 如果您之前使用的是 Azure 托管身份，现在需要改用构造函数中的 `azureADTokenProvider` 参数代替 `credentials`，更多细节请参见 [Azure 托管身份](#using-azure-managed-identity) 部分。

   - 如果您之前使用的是环境变量，现在需要设置 `AZURE_OPENAI_API_INSTANCE_NAME` 环境变量代替 `AZURE_OPENAI_API_ENDPOINT`，并新增 `AZURE_OPENAI_API_VERSION` 环境变量来指定 API 版本。


## API 参考文档

有关 AzureOpenAIEmbeddings 所有功能和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/langchain_openai.AzureOpenAIEmbeddings.html